<h1>DySARNet Trainer</h1>

In [ ]:
# LOAD DEPENDENCIES
import os
import cv2
import time
import glob
# import model
import pickle
import logging
import itertools
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import model_settings as ms
from utils.model import create_dysarnet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import applications, Model, layers

from tensorflow.keras.layers import Input
from matplotlib.ticker import FuncFormatter, PercentFormatter
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import (accuracy_score, 
                             mean_squared_error, 
                             mean_squared_log_error, 
                             classification_report, 
                             confusion_matrix, 
                             roc_curve, 
                             auc)

# PREVENT ERROR UNCESSARY MESSAGES
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
# Set the desired dataset to train and task to perform.
cwd = os.getcwd()
main_ds = os.path.join(cwd, 'ds')

task = ['binary', 'severity']
ds_folder = ['ua', 'torgo']

task_idx = 0  # Change to select binary or severity task
ds_folder_idx = 1  # Change to select 'torgo' dataset (0 for 'ua' dataset)

In [ ]:
# Loading the data.
selected_ds = os.path.join(main_ds, 
                           ds_folder[ds_folder_idx], 
                           f"{ds_folder[ds_folder_idx]}_{task[task_idx]}")
print(selected_ds)

train_data_dir = os.path.join(selected_ds, 'train')
validation_data_dir = os.path.join(selected_ds, 'val')
test_data_dir = os.path.join(selected_ds, 'test')

img_rows, img_cols = 224, 224
input_shape = (img_rows, img_cols, 3)
model_input = Input(shape=input_shape)

print("*"*55)
print("Data folders found!")
print("*"*55)
print(f"Train Path: {train_data_dir}")
print(f"Validation Path: {validation_data_dir}")
print(f"Test Path: {test_data_dir}")
print("*"*55)
print(f"The Input size is set to {img_rows} {img_cols} with a depth of {input_shape[-1]}.") 
print("*"*55)

In [ ]:
# Setting the architecture name depedning on the model to be trained.
architecture_name = "DySARNet"
model_name = architecture_name + '-' + task[task_idx].title() + '-' + ds_folder[ds_folder_idx].upper()
model_path = os.path.join('model/', model_name)

print(f"Model name: {model_name}")

In [ ]:
# Set hyper-parameters.
batch_size = 16
epochs = 10
optimizer = Adam(learning_rate=0.001)

In [ ]:
# Prepare data generators.
preprocessing = ImageDataGenerator(rescale = 1. / 255)

# Set class labels
if task_idx == 0:
    classes = ['0_Normal', '1_Dysarthria']
else:
    classes = ['0_Normal', '1_Mild', '2_Moderate', '3_SeverelyModerate', '4_Severe']

# Generators
train_datagen = preprocessing
         
val_datagen = preprocessing

test_datagen = preprocessing

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        seed=42,
        classes=classes)

validation_generator = val_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        seed=42,
        shuffle=False,
        classes=classes)

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_rows,img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        seed=42,
        shuffle=False,
        classes=classes)

# CHECK  THE NUMBER OF SAMPLES
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

print("Train samples:", nb_train_samples)
print("Validation samples:", nb_validation_samples)
print("Test samples:", nb_test_samples)
print(f"TOTAL SAMPLES: {(nb_train_samples + nb_validation_samples + nb_test_samples)}")

if nb_train_samples == 0:
    print("NO DATA TRAIN FOUND! Please check your train data path and folders!")
else:
    print("Train samples found!")
    
if nb_validation_samples == 0:
    print("NO DATA VALIDATION FOUND! Please check your validation data path and folders!")
    print("Check the data folders first!")
else:
    print("Validation samples found!")
    
if nb_test_samples == 0:
    print("NO DATA TEST FOUND! Please check your test data path and folders!")
    print("Check the data folders first!")
else:
    print("Test samples found!")

# check the class indices
train_generator.class_indices
validation_generator.class_indices
test_generator.class_indices

# true labels
Y_test=validation_generator.classes
test_labels = test_generator.classes

num_classes= len(train_generator.class_indices)

print('Model set to train', num_classes, 'classes')

if nb_train_samples and nb_validation_samples and nb_test_samples > 0:
    print("Generators are set!")
    print("Check if dataset is complete and has no problems before proceeding.")

In [ ]:
# Auto selects the activation based on the number of classes.

if num_classes > 2:
    class_activation="softmax"
elif num_classes <= 2:
    class_activation="sigmoid"

print(f"The class activation is set to {class_activation} due to the classes being {num_classes}.")
print(f"\nThe classes to be classified are {classes}")

In [ ]:
# Generates the DySARNet model.
model = create_dysarnet(input_shape=(224,224,3), 
                                        activation=ms.activation,
                                        growth_rate=ms.growth_rate, 
                                        bottleneck_size=ms.bottleneck_size, 
                                        nb_dense_block=ms.nb_dense_block, 
                                        nb_filter=ms.nb_filter,
                                        model_name=model_name,
                                        class_activation=class_activation,
                                        num_classes=num_classes)
model.summary()

In [ ]:
# Saving the model file
model_dir = 'model/'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)
    print('Model directory', model_dir, 'successfully created')
else:
    print('Model directory already exist, no new directory made.')

print()
print('-'*49)
print('Model directory is available for saving the model!')
print('-'*49)

In [ ]:
# Compile the model.
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy']) 

# Train the model.
start_time = time.time()
print(f"{model_name} started training!\n")

history = model.fit(train_generator, 
                    validation_data=validation_generator, 
                    steps_per_epoch=nb_train_samples // batch_size,
                    validation_steps=nb_validation_samples // batch_size,
                    epochs=epochs,
                    verbose=1)

elapsed_time = time.time() - start_time
train_time = time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
print()
print("Training Time:")
print("--------------")
print(f"Elapsed time in seconds: {elapsed_time:.2f} seconds")
print(f"Elapsed time in minutes: {elapsed_time / 60:.2f} minutes")
print(f"Elapsed time in hours: {elapsed_time / 3600:.2f} hours")
print()
print(f"Total Training Time: {train_time}")


In [ ]:
# Perform initial validations
val_scores = model.evaluate(validation_generator, return_dict=True, verbose=1)

val_acc = val_scores['accuracy'] * 100
val_loss = val_scores['loss'] * 100

print(f"\nValidation Results of {model_name}")
print(f"Val accuracy: {val_acc:.2f}%")
print(f"Val loss: {val_loss:.2f}%")


In [ ]:
# Perform initial testing
test_scores = model.evaluate(test_generator, return_dict=True, verbose=1)

test_acc = test_scores['accuracy'] * 100
test_loss = test_scores['loss'] * 100

print(f"\nTest Results of {model_name}")
print(f"Test accuracy: {test_acc:.2f}%")
print(f"Test loss: {test_loss:.2f}%")

In [ ]:
# Save the model if satisfied.
print(f"MODEL {model_name} SERIALIZING WAIT FOR A MOMENT...")
print()
model.save('model/' + model._name + '/' + model._name + "-" + f"{test_acc:.2f}" + '.h5')
print(f"Model serialized as {model_name}")

In [ ]:
# Save the history for future evaluation.
def save_h(file, history):
    with open(file + '/' + model._name + '/' + model_name + "-" + f"{test_acc:.2f}" + '.history', 'wb') as file_pi:
        pickle.dump(history, file_pi)
    print(f"{model._name} history saved!")

save_h('model/', history.history)

print()
print()
print(f"The Model weights and history of {model._name} are successfully saved!")

In [ ]:
# Generate predictions from the validation set.
y_pred = model.predict(validation_generator, 
                       nb_validation_samples/batch_size, 
                       workers=1, 
                       verbose=1)

In [ ]:
# Generate predictions from the test set.
test_pred = model.predict(test_generator, 
                          nb_validation_samples/batch_size, 
                          workers=1, 
                          verbose=1)

In [ ]:
# Take the test accuracy percentage to name the model
test_acc = test_scores['accuracy']
accuracy_percentage = "{:.2%}".format(test_acc)

print(f'The model achieved a test accuracy of: {accuracy_percentage}')